In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def getConserved(rho, vx, vy, P, gamma, vol):
    """
    Calculate the conserved variables from the primitives.
    """
    Mass   = rho * vol
    Momx   = rho * vx * vol
    Momy   = rho * vy * vol
    Energy = (P/(gamma-1) + 0.5*rho*(vx**2+vy**2)) * vol
    return Mass, Momx, Momy, Energy

def setGhostCells(rho, vx, vy, P):
    """
    Set ghost cells at the boundaries (top and bottom).
    """
    rho[:, 0]  = rho[:, 1]
    vx[:, 0]   = vx[:, 1]
    vy[:, 0]   = -vy[:, 1]
    P[:, 0]    = P[:, 1]
    
    rho[:, -1] = rho[:, -2]
    vx[:, -1]  = vx[:, -2]
    vy[:, -1]  = -vy[:, -2]
    P[:, -1]   = P[:, -2]
    return rho, vx, vy, P

def getPrimitive(Mass, Momx, Momy, Energy, gamma, vol):
    """
    Calculate the primitive variables from the conserved variables.
    """
    rho = Mass / vol
    vx  = Momx / rho / vol
    vy  = Momy / rho / vol
    P   = (Energy/vol - 0.5*rho*(vx**2+vy**2)) * (gamma-1)
    rho, vx, vy, P = setGhostCells(rho, vx, vy, P)
    return rho, vx, vy, P

def getGradient(f, dx):
    """
    Calculate gradients using central differences.
    """
    R = -1  # right
    L = 1   # left
    f_dx = (np.roll(f, R, axis=0) - np.roll(f, L, axis=0)) / (2 * dx)
    f_dy = (np.roll(f, R, axis=1) - np.roll(f, L, axis=1)) / (2 * dx)
    f_dx, f_dy = setGhostGradients(f_dx, f_dy)
    return f_dx, f_dy

def slopeLimit(f, dx, f_dx, f_dy):
    """
    Apply a slope limiter to the computed gradients.
    """
    R = -1  # right
    L = 1   # left
    f_dx = np.maximum(0., np.minimum(1., ((f - np.roll(f, L, axis=0)) / dx) / (f_dx + 1.0e-8 * (f_dx==0)))) * f_dx
    f_dx = np.maximum(0., np.minimum(1., (-(f - np.roll(f, R, axis=0)) / dx) / (f_dx + 1.0e-8 * (f_dx==0)))) * f_dx
    f_dy = np.maximum(0., np.minimum(1., ((f - np.roll(f, L, axis=1)) / dx) / (f_dy + 1.0e-8 * (f_dy==0)))) * f_dy
    f_dy = np.maximum(0., np.minimum(1., (-(f - np.roll(f, R, axis=1)) / dx) / (f_dy + 1.0e-8 * (f_dy==0)))) * f_dy
    return f_dx, f_dy

def setGhostGradients(f_dx, f_dy):
    """
    Set ghost cell gradients to be reflective at the boundaries.
    """
    f_dy[:, 0]  = -f_dy[:, 1]
    f_dy[:, -1] = -f_dy[:, -2]
    return f_dx, f_dy

def extrapolateInSpaceToFace(f, f_dx, f_dy, dx):
    """
    Extrapolate the cell-centered values to the face centers.
    """
    R = -1  # right
    f_XL = f - f_dx * dx/2
    f_XL = np.roll(f_XL, R, axis=0)
    f_XR = f + f_dx * dx/2
    f_YL = f - f_dy * dx/2
    f_YL = np.roll(f_YL, R, axis=1)
    f_YR = f + f_dy * dx/2
    return f_XL, f_XR, f_YL, f_YR

def applyFluxes(F, flux_F_X, flux_F_Y, dx, dt):
    """
    Update the conserved variable using the computed fluxes.
    """
    L = 1   # left
    F += - dt * dx * flux_F_X
    F +=   dt * dx * np.roll(flux_F_X, L, axis=0)
    F += - dt * dx * flux_F_Y
    F +=   dt * dx * np.roll(flux_F_Y, L, axis=1)
    return F

def getFlux(rho_L, rho_R, vx_L, vx_R, vy_L, vy_R, P_L, P_R, gamma):
    """
    Compute the fluxes between left and right states using a local Rusanov solver.
    """
    en_L = P_L/(gamma-1) + 0.5 * rho_L * (vx_L**2 + vy_L**2)
    en_R = P_R/(gamma-1) + 0.5 * rho_R * (vx_R**2 + vy_R**2)
    
    # Star (averaged) states
    rho_star  = 0.5 * (rho_L + rho_R)
    momx_star = 0.5 * (rho_L * vx_L + rho_R * vx_R)
    momy_star = 0.5 * (rho_L * vy_L + rho_R * vy_R)
    en_star   = 0.5 * (en_L + en_R)
    
    P_star = (gamma-1) * (en_star - 0.5*(momx_star**2 + momy_star**2) / rho_star)
    
    # Compute fluxes
    flux_Mass   = momx_star
    flux_Momx   = momx_star**2 / rho_star + P_star
    flux_Momy   = momx_star * momy_star / rho_star
    flux_Energy = (en_star + P_star) * momx_star / rho_star
    
    # Determine wavespeeds
    C_L = np.sqrt(gamma * P_L / rho_L) + np.abs(vx_L)
    C_R = np.sqrt(gamma * P_R / rho_R) + np.abs(vx_R)
    C = np.maximum(C_L, C_R)
    
    # Add the diffusive term
    flux_Mass   -= C * 0.5 * (rho_L - rho_R)
    flux_Momx   -= C * 0.5 * (rho_L * vx_L - rho_R * vx_R)
    flux_Momy   -= C * 0.5 * (rho_L * vy_L - rho_R * vy_R)
    flux_Energy -= C * 0.5 * (en_L - en_R)
    
    return flux_Mass, flux_Momx, flux_Momy, flux_Energy

def addGhostCells(rho, vx, vy, P):
    """
    Add ghost cells on the left and right boundaries.
    """
    rho = np.hstack((rho[:, 0:1], rho, rho[:, -1:]))
    vx  = np.hstack((vx[:, 0:1], vx, vx[:, -1:]))
    vy  = np.hstack((vy[:, 0:1], vy, vy[:, -1:]))
    P   = np.hstack((P[:, 0:1], P, P[:, -1:]))
    return rho, vx, vy, P

def addSourceTerm(Mass, Momx, Momy, Energy, g, dt):
    """
    Add a gravitational source term to the conserved variables.
    """
    Energy += dt * Momy * g
    Momy   += dt * Mass * g
    return Mass, Momx, Momy, Energy

def main():
    # Simulation parameters
    N                = 128       # resolution: N x 3N grid
    boxsizeX         = 0.5
    boxsizeY         = 1.5
    gamma            = 1.4      # ideal gas gamma
    courant_fac      = 0.4
    t                = 0
    tEnd             = 15
    tOut             = 0.1      # output interval (simulation time)
    useSlopeLimiting = False
    plotRealTime     = True
    
    # Mesh generation
    dx = boxsizeX / N
    vol = dx**2
    xlin = np.linspace(0.5 * dx, boxsizeX - 0.5 * dx, N)
    ylin = np.linspace(0.5 * dx, boxsizeY - 0.5 * dx, 3 * N)
    Y, X = np.meshgrid(ylin, xlin)
    
    # Initial conditions: heavy fluid on top of light fluid with a small perturbation.
    g_val = -0.1  # gravity
    w0 = 0.0025
    P0 = 2.5
    rho = 1. + (Y > 0.75)
    vx  = np.zeros(X.shape)
    vy  = w0 * (1 - np.cos(4 * np.pi * X)) * (1 - np.cos(4 * np.pi * Y / 3))
    P   = P0 + g_val * (Y - 0.75) * rho
    
    # Add ghost cells on the left and right boundaries.
    rho, vx, vy, P = addGhostCells(rho, vx, vy, P)
    
    # Compute the conserved variables.
    Mass, Momx, Momy, Energy = getConserved(rho, vx, vy, P, gamma, vol)
    
    # Prepare the figure for plotting.
    fig = plt.figure(figsize=(4,4), dpi=80)
    
    # Estimate the total number of frames that will be produced.
    total_frames_estimate = int(tEnd / tOut) + 1
    desired_saved_frames = 6
    # Compute indices (0-based) at which to save the frames (equally spaced).
    save_indices = np.linspace(0, total_frames_estimate - 1, desired_saved_frames)
    save_indices = set(np.round(save_indices).astype(int))
    print("Will save frames with indices:", sorted(save_indices))
    
    outputCount = 1  # counts the number of frames drawn (starting at 1)
    
    # Main simulation loop.
    while t < tEnd:
        # Update primitive variables from conserved variables.
        rho, vx, vy, P = getPrimitive(Mass, Momx, Momy, Energy, gamma, vol)
        
        # Compute the time step based on the CFL condition.
        dt = courant_fac * np.min(dx / (np.sqrt(gamma * P/rho) + np.sqrt(vx**2 + vy**2)))
        plotThisTurn = False
        if t + dt > outputCount * tOut:
            dt = outputCount * tOut - t
            plotThisTurn = True
        
        # Half-step source term update.
        Mass, Momx, Momy, Energy = addSourceTerm(Mass, Momx, Momy, Energy, g_val, dt/2)
        
        # Update primitives after source term.
        rho, vx, vy, P = getPrimitive(Mass, Momx, Momy, Energy, gamma, vol)
        
        # Compute gradients.
        rho_dx, rho_dy = getGradient(rho, dx)
        vx_dx, vx_dy   = getGradient(vx, dx)
        vy_dx, vy_dy   = getGradient(vy, dx)
        P_dx, P_dy     = getGradient(P, dx)
        
        # Optionally apply slope limiting.
        if useSlopeLimiting:
            rho_dx, rho_dy = slopeLimit(rho, dx, rho_dx, rho_dy)
            vx_dx, vx_dy   = slopeLimit(vx, dx, vx_dx, vx_dy)
            vy_dx, vy_dy   = slopeLimit(vy, dx, vy_dx, vy_dy)
            P_dx, P_dy     = slopeLimit(P, dx, P_dx, P_dy)
        
        # Extrapolate half-step in time.
        rho_prime = rho - 0.5 * dt * (vx * rho_dx + rho * vx_dx + vy * rho_dy + rho * vy_dy)
        vx_prime  = vx  - 0.5 * dt * (vx * vx_dx + vy * vx_dy + (1/rho) * P_dx)
        vy_prime  = vy  - 0.5 * dt * (vx * vy_dx + vy * vy_dy + (1/rho) * P_dy)
        P_prime   = P   - 0.5 * dt * (gamma * P * (vx_dx + vy_dy) + vx * P_dx + vy * P_dy)
        
        # Extrapolate from cell centers to faces.
        rho_XL, rho_XR, rho_YL, rho_YR = extrapolateInSpaceToFace(rho_prime, rho_dx, rho_dy, dx)
        vx_XL, vx_XR, vx_YL, vx_YR     = extrapolateInSpaceToFace(vx_prime, vx_dx, vx_dy, dx)
        vy_XL, vy_XR, vy_YL, vy_YR     = extrapolateInSpaceToFace(vy_prime, vy_dx, vy_dy, dx)
        P_XL, P_XR, P_YL, P_YR         = extrapolateInSpaceToFace(P_prime, P_dx, P_dy, dx)
        
        # Compute fluxes using the Rusanov (local Lax-Friedrichs) method.
        flux_Mass_X, flux_Momx_X, flux_Momy_X, flux_Energy_X = getFlux(rho_XL, rho_XR, vx_XL, vx_XR, vy_XL, vy_XR, P_XL, P_XR, gamma)
        flux_Mass_Y, flux_Momy_Y, flux_Momx_Y, flux_Energy_Y = getFlux(rho_YL, rho_YR, vy_YL, vy_YR, vx_YL, vx_YR, P_YL, P_YR, gamma)
        
        # Update conserved variables using the computed fluxes.
        Mass   = applyFluxes(Mass, flux_Mass_X, flux_Mass_Y, dx, dt)
        Momx   = applyFluxes(Momx, flux_Momx_X, flux_Momx_Y, dx, dt)
        Momy   = applyFluxes(Momy, flux_Momy_X, flux_Momy_Y, dx, dt)
        Energy = applyFluxes(Energy, flux_Energy_X, flux_Energy_Y, dx, dt)
        
        # Final half-step source term update.
        Mass, Momx, Momy, Energy = addSourceTerm(Mass, Momx, Momy, Energy, g_val, dt/2)
        
        # Update simulation time.
        t += dt
        
        # Plot the current frame.
        if (plotRealTime and plotThisTurn) or (t >= tEnd):
            plt.cla()
            plt.imshow(rho.T)
            plt.clim(0.8, 2.2)
            ax = plt.gca()
            ax.invert_yaxis()
            ax.get_xaxis().set_visible(False)
            ax.get_yaxis().set_visible(False)
            ax.set_aspect('equal')
            plt.pause(0.001)
            
            # If the current (zero-based) frame index is in our save list,
            # save the frame as an individual PDF.
            if (outputCount - 1) in save_indices:
                filename = f"frame_{outputCount - 1:03d}.pdf"
                plt.savefig(filename, format="pdf")
                print(f"Saved {filename}")
            outputCount += 1
    
    # Optionally save the final image as a PNG.
    plt.savefig('finitevolume2.png', dpi=240)
    plt.show()
    
    return 0

if __name__ == "__main__":
    main()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

# ----------------------------
# Simulation & Physical Parameters
# ----------------------------
nx, ny = 128, 128         # Number of grid cells in x and y
Lx, Ly = 1.0, 1.0         # Domain dimensions
dx = Lx / nx              # Grid spacing (assume dx = dy)
dt = 0.0005               # Time step
nsteps = 5000             # Total number of time steps
plot_interval = 50        # Steps between animation frames

# Physical parameters
nu = 1e-4                 # Kinematic viscosity
D  = 1e-4                 # Diffusion coefficient for density
g = 9.81                  # Gravitational acceleration (assume y increases upward)
# For Rayleigh–Taylor we need a density inversion: heavy fluid above light fluid.
rho_light = 1.0
rho_heavy = 2.0
rho0 = 0.5 * (rho_light + rho_heavy)  # reference density

# ----------------------------
# Grid & Initial Conditions
# ----------------------------
x = np.linspace(0, Lx, nx, endpoint=False)
y = np.linspace(0, Ly, ny, endpoint=False)
X, Y = np.meshgrid(x, y, indexing='ij')

# Define a perturbed interface. Here, cells with Y above the interface are assigned the heavy fluid.
perturbation_amplitude = 0.001
interface = Ly/2 + perturbation_amplitude * np.cos(2*np.pi*X/Lx)
rho = np.where(Y > interface, rho_heavy, rho_light)

# Initialize velocity fields (u: horizontal, v: vertical) and pressure
u = np.zeros((nx, ny))
v = np.zeros((nx, ny))
p = np.zeros((nx, ny))

# ----------------------------
# Helper Functions: Finite Volume Operators
# ----------------------------
def laplacian(f):
    """
    Computes the Laplacian of f using central differences
    on a uniform grid with periodic boundaries.
    """
    return (np.roll(f, -1, axis=0) + np.roll(f, 1, axis=0) +
            np.roll(f, -1, axis=1) + np.roll(f, 1, axis=1) - 4*f) / (dx*dx)

# ----------------------------
# Time-stepping: Projection Method
# ----------------------------
def update_simulation(rho, u, v, p):
    # --- Convective Terms ---
    # Use central differences (finite volume flux differences) for advection.
    u_conv = u * (np.roll(u, -1, axis=0) - np.roll(u, 1, axis=0)) / (2*dx) \
           + v * (np.roll(u, -1, axis=1) - np.roll(u, 1, axis=1)) / (2*dx)
    v_conv = u * (np.roll(v, -1, axis=0) - np.roll(v, 1, axis=0)) / (2*dx) \
           + v * (np.roll(v, -1, axis=1) - np.roll(v, 1, axis=1)) / (2*dx)
    
    # --- Diffusion Terms ---
    u_diff = nu * laplacian(u)
    v_diff = nu * laplacian(v)
    
    # --- Buoyancy (only in vertical momentum equation) ---
    # Under Boussinesq, buoyancy force is proportional to density deviation.
    # With gravity acting downward (negative y-direction), we use:
    F_y = -g * (rho - rho0) / rho0

    # --- Intermediate Velocities (u*, v*) ---
    u_star = u + dt * (-u_conv + u_diff)
    v_star = v + dt * (-v_conv + v_diff + F_y)
    
    # --- Pressure Projection ---
    # Compute divergence of the intermediate velocity field.
    div_u = (np.roll(u_star, -1, axis=0) - np.roll(u_star, 1, axis=0)) / (2*dx) + \
            (np.roll(v_star, -1, axis=1) - np.roll(v_star, 1, axis=1)) / (2*dx)
    # Pressure Poisson RHS: (divergence)/dt
    rhs = div_u / dt
    
    # Solve Laplace(p) = rhs using Jacobi iteration.
    p_new = np.copy(p)
    for _ in range(50):  # 50 Jacobi iterations (can be increased for better accuracy)
        p_new = 0.25 * (np.roll(p_new, -1, axis=0) + np.roll(p_new, 1, axis=0) +
                        np.roll(p_new, -1, axis=1) + np.roll(p_new, 1, axis=1) - rhs * (dx*dx))
    p = p_new
    
    # Correct velocities to enforce incompressibility.
    dpdx = (np.roll(p, -1, axis=0) - np.roll(p, 1, axis=0)) / (2*dx)
    dpdy = (np.roll(p, -1, axis=1) - np.roll(p, 1, axis=1)) / (2*dx)
    u_new = u_star - dt * dpdx
    v_new = v_star - dt * dpdy
    
    # --- Density Update ---
    # Advection of density using central differences and add diffusion.
    rho_conv = u * (np.roll(rho, -1, axis=0) - np.roll(rho, 1, axis=0)) / (2*dx) \
             + v * (np.roll(rho, -1, axis=1) - np.roll(rho, 1, axis=1)) / (2*dx)
    rho_diff = D * laplacian(rho)
    rho_new = rho + dt * (-rho_conv + rho_diff)
    
    return rho_new, u_new, v_new, p

# ----------------------------
# Set Up Animation
# ----------------------------
fig, ax = plt.subplots(figsize=(16, 12))
im = ax.imshow(rho, extent=[0, Lx, 0, Ly], origin='lower', cmap='viridis')
ax.set_title('Rayleigh–Taylor Instability (Finite Volume)')
ax.set_xlabel('x')
ax.set_ylabel('y')
plt.colorbar(im, ax=ax)

def animate(frame):
    global rho, u, v, p
    # Advance simulation a few steps per frame for a visible evolution.
    for _ in range(plot_interval):
        rho, u, v, p = update_simulation(rho, u, v, p)
    im.set_data(rho)
    ax.set_title(f'RT Instability (Finite Volume) at t = {frame*plot_interval*dt:.3f}')
    return [im]

ani = animation.FuncAnimation(fig, animate, frames=200, interval=30, blit=True)
# Save the animation as a GIF
ani.save('rayleigh_taylor_instability.mp4', writer='imagemagick', fps=20, dpi=240)
plt.show()
